In [1]:
import scipy.io as reader
from scipy.spatial.distance import cdist
import numpy as np

In [2]:
#loading data

train = reader.loadmat('../../../../Results/5crops/VGG16(+pca -robustS -standardS)/train_VGG16TrueFalseFalse.mat')['data']
test = reader.loadmat('../../../../Results/5crops/VGG16(+pca -robustS -standardS)/test_VGG16TrueFalseFalse.mat')['data']

trainAnnotation = reader.loadmat('../../AnotationMatrices/trainAnnotationFlickr25k.mat')['trainAnnotation']
testAnnotation = reader.loadmat('../../AnotationMatrices/testAnnotationFlickr25k.mat')['testAnnotation']

print(testAnnotation.shape)

(12500, 38)


In [3]:
print(train.shape)

(12500, 2318)


In [4]:
#parameters

labels=5
k=4
sigma=15

In [5]:
distMatrix_train2test = cdist(train, test)

In [6]:
distMatrix_train2test.shape

(12500, 12500)

In [7]:
def twopassknn2(distance_vI_vJ,trainAnnotation,testAnnotation,K1,w):
    numOfTestImages = testAnnotation.shape[0] # 12500 test img
    numOfTrainImages = trainAnnotation.shape[0] # 12500 train img
    
    numOfLabels = trainAnnotation.shape[1] # 38 label
    labelTableSize = np.zeros((numOfLabels,1)) # 38x1
    
    labelTableSize = np.sum(trainAnnotation, axis=0) # har label barash chanta aks darim dar train
    
    Score_w_I_R = np.zeros((numOfLabels,numOfTestImages)) # 38x12500
    
    for i in range(numOfTestImages):
        subsetTrain = np.zeros((numOfLabels*K1,1)) # 38*4 -> 152
        subsetCitation = np.zeros((numOfLabels*K1,1)) # 38*4 -> 152
        
        # 1st knn pass
        count = 0;
        for j in range(numOfLabels):
            currTrainSet = np.array( np.where(trainAnnotation[:,j]==1) ) # idx axayi ke label j ra darand
            currTrainIndxDist = np.concatenate([currTrainSet , distance_vI_vJ[currTrainSet,i]]).T # currTrainSet + dis(i, currTrainSet) -> [idx dist]
            currTrainIndxDist = currTrainIndxDist[currTrainIndxDist[:,1].argsort()] # sort by dist
            
            for k in range(K1): # K: 1 -> 4
                if( k <= labelTableSize[j] ):
                    subsetTrain[count] = currTrainIndxDist[k][0]; #subsetTrain [1...k1    0 0 0 0  ...] -> [idx(1nn be l1) idx(2nn be l1) idx(3nn be l1) idx(4nn be l1) - idx(1nn be l2) ... idx(4nn be l38)]
                                                                 #knn idx bara[label(1)  label(2) ...]
                    count = count + 1
                else:
                    break

        
        
        subsetTrain = subsetTrain[0:count]
        subsetTrain = np.array([np.unique(subsetTrain)]).T
        temp = np.concatenate([subsetTrain.astype('uint64'),  np.array([distance_vI_vJ[subsetTrain[:,0].astype('uint64'),i]]).T], axis=1) # k1 nn be label(1) + dis(i, currTrainSet) -> [idx dist] -> 152x2
        
    
        minDist = np.min(temp[:,1])
        meanDist = np.mean(temp[:,1])
        
        # 2nd knn pass
        temp = temp[temp[:,1].argsort()] # sort by dist -> qaroqaT mishe idx ha
        subsetTrain = temp[:,0]
        currScores = np.zeros((numOfLabels,1)) # 38x1
        labelFreq = np.zeros((numOfLabels,1)) # 38x1
        pickNbrs = subsetTrain.shape[0] # 4*38 -> max = 152 -> (<152)
        
        for j in range(pickNbrs):
            trainIndx = int(subsetTrain[j])
            currLabels = np.array( np.where(trainAnnotation[trainIndx,:]==1) )[0] # label hayi ke har yek az 152 ta ax tush hastan
            dist1 = distance_vI_vJ[trainIndx,i] # dist(har yek az 152, axe test)
            distVal = ((dist1-minDist)/(meanDist)) #normilization
            val = np.exp(-w*distVal) # w=15
            for l in range(len(currLabels)):
                currScores[currLabels[l]] += val
                labelFreq[currLabels[l]] += 1
        
        currScores = currScores/np.sum(currScores)
        Score_w_I_R[:,i] = currScores[:, 0]
        
    
    # Normalize 
    for i in range(numOfLabels):
        maxx = np.max(Score_w_I_R[i,:])
        if( maxx>0 ):            
            Score_w_I_R[i,:] = Score_w_I_R[i,:]/maxx;
    
    return Score_w_I_R

In [8]:
Score_w_I = twopassknn2(distMatrix_train2test,trainAnnotation,testAnnotation,k,sigma)

In [78]:
# Save scores mat
reader.savemat('./scores.mat', {'dict': Score_w_I})

In [11]:
# Maybe you need the copy later! :|
temp = np.copy(Score_w_I)
print(Score_w_I)

[[0.04035128 0.02647352 0.01875159 ... 0.04324425 0.0154889  0.02592783]
 [0.2997902  0.1556756  0.11176736 ... 0.20148876 0.04291744 0.41407898]
 [0.15552348 0.06818446 0.04206506 ... 0.13902142 0.02290399 0.16773473]
 ...
 [0.14876734 0.16664018 0.30725657 ... 0.20786943 0.26356012 0.14270241]
 [0.03808868 0.05981837 0.08140029 ... 0.04588089 0.04307497 0.03471642]
 [0.15644749 0.20503698 0.16540933 ... 0.11781442 0.25101727 0.11503915]]


In [9]:
def computePerf(testAnnotations,Score,annotLabels):
    numOfTestImages = testAnnotations.shape[0]
    numOfLabels = testAnnotations.shape[1]
    testLabelTableSize = np.zeros((numOfLabels,1))
    
    testLabelTableSize = np.sum(testAnnotations, axis=0) # har label barash chanta aks darim dar train
    
    # precision, recall, f1-score, nplus 

    correct = np.zeros((numOfLabels,1))
    predict = np.zeros((numOfLabels,1))
    ground = np.zeros((numOfLabels,1))
    
    for i in range(numOfTestImages):
        
        actualLabels = np.array( np.where(testAnnotations[i, :]==1) )
        currScores1 = Score[:,i]
        assignedLabels = np.zeros((1,annotLabels))
        for j in range(annotLabels):
            val,indx = np.max(currScores1), np.argmax(currScores1)
            assignedLabels[0][j] = indx
            currScores1[indx] = -np.inf;
        
        for j in range(assignedLabels.shape[1]):
            predict[int(assignedLabels[0][j])] += 1

        for j in range(actualLabels.shape[1]):
            ground[int(actualLabels[0][j])] += 1
            for k in range(assignedLabels.shape[1]):
                if( assignedLabels[0][k]==actualLabels[0][j] ):
                    correct[int(assignedLabels[0][k])] += 1
        print('image test number: ', i)
        print('actual labels: ', actualLabels)
        print('assign labels: ', assignedLabels)        
                        
    prec = np.zeros((numOfLabels,1))[:, 0]
    rec = np.zeros((numOfLabels,1))[:, 0]
    for i in range(numOfLabels):
        if predict[i]>0:
            prec[i] = (correct[i]/predict[i])[0]
        if ground[i]>0:
            rec[i] = (correct[i]/ground[i])[0]
            
    
    precision = np.mean(prec)
    recall = np.mean(rec)
    f1score = 2*precision*recall/(precision+recall)
    nplus = np.array( np.where(rec>0) )

    print([precision, recall, f1score, nplus.shape[1]])
    return [precision, recall, f1score, nplus.shape[1]]

    

In [10]:
performance = computePerf(testAnnotation,np.copy(Score_w_I),labels)

image test number:  0
actual labels:  [[24]]
assign labels:  [[23. 22. 19. 18. 26.]]
image test number:  1
actual labels:  [[18 19 22 23 29 31 33 34 37]]
assign labels:  [[29. 23. 22. 19. 18.]]
image test number:  2
actual labels:  [[ 7 17 27 31 32 33 37]]
assign labels:  [[35. 29.  7. 32. 34.]]
image test number:  3
actual labels:  [[24 37]]
assign labels:  [[24.  0. 37. 33. 13.]]
image test number:  4
actual labels:  []
assign labels:  [[35. 23. 22. 24. 12.]]
image test number:  5
actual labels:  [[11 12 16 22 23 25 26]]
assign labels:  [[26. 25. 23. 12. 11.]]
image test number:  6
actual labels:  [[24 27 37]]
assign labels:  [[27. 35.  7. 37. 28.]]
image test number:  7
actual labels:  [[11 18 20 21 22 23 31 32]]
assign labels:  [[33. 35. 20. 22. 23.]]
image test number:  8
actual labels:  [[20 32]]
assign labels:  [[22. 23. 32.  7. 18.]]
image test number:  9
actual labels:  [[ 7 31]]
assign labels:  [[31.  7. 32. 35. 33.]]
image test number:  10
actual labels:  [[ 0 13 24]]
assign

image test number:  316
actual labels:  [[20 22]]
assign labels:  [[23. 22. 26. 25. 12.]]
image test number:  317
actual labels:  [[11 12 22 23 24 25 26 33 35]]
assign labels:  [[26. 25. 23. 12. 22.]]
image test number:  318
actual labels:  [[11 12 22 23 32]]
assign labels:  [[23. 19. 22. 18.  5.]]
image test number:  319
actual labels:  [[ 1  2 11 12 16 22 23 25 26]]
assign labels:  [[26. 25. 23. 22. 12.]]
image test number:  320
actual labels:  [[11 12 16 18 19 20 22 23]]
assign labels:  [[23. 22. 11. 18. 20.]]
image test number:  321
actual labels:  [[11 12 16 22 23 25 26]]
assign labels:  [[23. 35. 26. 25. 22.]]
image test number:  322
actual labels:  [[13 14 24]]
assign labels:  [[13. 24. 14. 22. 33.]]
image test number:  323
actual labels:  [[11 16 22 23]]
assign labels:  [[25. 26. 23. 12. 22.]]
image test number:  324
actual labels:  [[16]]
assign labels:  [[23. 22. 18. 26. 25.]]
image test number:  325
actual labels:  [[11 16 18 22 32]]
assign labels:  [[23. 22. 11. 18. 32.]]
i

image test number:  572
actual labels:  [[18 22 25 32]]
assign labels:  [[23. 22. 19. 26. 25.]]
image test number:  573
actual labels:  [[ 1 16 18 19 22 23 25 26]]
assign labels:  [[23. 22. 19. 18. 26.]]
image test number:  574
actual labels:  [[ 7 24 31 32 33 34 35]]
assign labels:  [[33. 35.  7. 24. 31.]]
image test number:  575
actual labels:  [[16 18 19 22 23 24]]
assign labels:  [[23. 19. 22. 18. 35.]]
image test number:  576
actual labels:  [[13 14 24]]
assign labels:  [[24. 33. 13. 14.  7.]]
image test number:  577
actual labels:  [[11 12 16 22 23 25 26]]
assign labels:  [[25. 26. 23. 12. 22.]]
image test number:  578
actual labels:  [[11 16 22]]
assign labels:  [[23. 22. 19. 18. 26.]]
image test number:  579
actual labels:  [[ 7  8 31 32]]
assign labels:  [[ 7.  8. 35. 31. 32.]]
image test number:  580
actual labels:  [[ 7  8 31 33]]
assign labels:  [[33.  7. 35. 17.  8.]]
image test number:  581
actual labels:  [[ 7 24 31 32]]
assign labels:  [[35. 24.  7. 33. 23.]]
image test

assign labels:  [[33. 29.  7. 31. 17.]]
image test number:  856
actual labels:  [[16]]
assign labels:  [[22. 23. 11. 18. 19.]]
image test number:  857
actual labels:  [[24 32]]
assign labels:  [[35. 24. 23. 22. 32.]]
image test number:  858
actual labels:  [[16 18 19 22 23 25 26]]
assign labels:  [[23. 26. 25. 22. 19.]]
image test number:  859
actual labels:  [[24 31 33 35 37]]
assign labels:  [[23. 33. 22. 29.  7.]]
image test number:  860
actual labels:  [[16 18 19 22 23 25 26]]
assign labels:  [[23. 22. 26. 25. 18.]]
image test number:  861
actual labels:  [[ 0  3  4 24 35]]
assign labels:  [[35. 24.  3.  4. 23.]]
image test number:  862
actual labels:  [[ 0 32]]
assign labels:  [[24. 23. 22. 29. 35.]]
image test number:  863
actual labels:  [[13 16 24]]
assign labels:  [[23. 22. 24. 16. 11.]]
image test number:  864
actual labels:  [[ 7  8 31 34]]
assign labels:  [[35.  7. 29. 33.  8.]]
image test number:  865
actual labels:  [[11 12 16 18 19 22 23]]
assign labels:  [[23. 25. 26. 1

actual labels:  [[24 31 37]]
assign labels:  [[29. 30. 35. 37. 33.]]
image test number:  1156
actual labels:  [[ 0  3  4  7 31 37]]
assign labels:  [[17. 23. 35. 29. 27.]]
image test number:  1157
actual labels:  [[0 9]]
assign labels:  [[23. 22. 19. 12. 18.]]
image test number:  1158
actual labels:  [[ 0  9 10 32]]
assign labels:  [[23. 22. 19. 18. 35.]]
image test number:  1159
actual labels:  [[16 18 19 20 22 23 25 26]]
assign labels:  [[23. 26. 25.  2. 19.]]
image test number:  1160
actual labels:  [[16 22]]
assign labels:  [[23. 22. 12. 11. 18.]]
image test number:  1161
actual labels:  [[ 5  7 18 19 22 23 24 31 32 33 34 35]]
assign labels:  [[33.  7. 35. 31. 23.]]
image test number:  1162
actual labels:  [[ 5  6 11 12 22 24 32 34 35 36]]
assign labels:  [[35. 34. 23.  5. 22.]]
image test number:  1163
actual labels:  [[16 18 19 22 23 25 26]]
assign labels:  [[23. 25. 26. 22. 19.]]
image test number:  1164
actual labels:  [[16 18 19 20 22 23]]
assign labels:  [[23. 19. 20. 17. 22.

...

image test number:  11268
actual labels:  [[11 12 22 23 25 26 29 31 37]]
assign labels:  [[29. 23. 22. 30. 12.]]
image test number:  11269
actual labels:  [[ 5 24 32 34 35]]
assign labels:  [[35.  5. 22. 24. 34.]]
image test number:  11270
actual labels:  [[16 18 19 22 23]]
assign labels:  [[23. 22. 18. 12. 11.]]
image test number:  11271
actual labels:  [[ 7 18 22 23 29 31 34 37]]
assign labels:  [[29. 37. 31.  7. 34.]]
image test number:  11272
actual labels:  [[ 0 16 20]]
assign labels:  [[23. 22. 25. 26. 11.]]
image test number:  11273
actual labels:  [[18 19 22 23 25 26 32]]
assign labels:  [[25. 26. 23. 22. 19.]]
image test number:  11274
actual labels:  [[11 12 16 22 23 25 26]]
assign labels:  [[23. 22. 12. 26. 25.]]
image test number:  11275
actual labels:  [[ 7  8 18 19 22 23 24 27 31 37]]
assign labels:  [[ 7.  8. 35. 17. 33.]]
image test number:  11276
actual labels:  [[11 12 13 22 23 24 25 26 32]]
assign labels:  [[26. 25. 23. 12. 22.]]
image test number:  11277
actual labe

assign labels:  [[23. 22. 18. 19. 11.]]
image test number:  11561
actual labels:  [[0]]
assign labels:  [[23. 26. 25. 22. 12.]]
image test number:  11562
actual labels:  [[11 18 22 31 32]]
assign labels:  [[23. 11. 22. 37. 29.]]
image test number:  11563
actual labels:  [[ 7  8 24 29 31 35 37]]
assign labels:  [[ 8.  7. 35. 24. 27.]]
image test number:  11564
actual labels:  []
assign labels:  [[24. 23. 22. 35. 18.]]
image test number:  11565
actual labels:  [[ 7 24 27 31 32 33 35 36 37]]
assign labels:  [[33. 35. 36. 24. 17.]]
image test number:  11566
actual labels:  [[16]]
assign labels:  [[22. 23. 16. 12. 11.]]
image test number:  11567
actual labels:  [[18 19 22 23 25 26 32]]
assign labels:  [[26. 25. 22. 23. 18.]]
image test number:  11568
actual labels:  [[18 22 24 32]]
assign labels:  [[35. 24. 23. 22. 11.]]
image test number:  11569
actual labels:  [[13 14 24]]
assign labels:  [[24.  7. 33. 23. 35.]]
image test number:  11570
actual labels:  [[32]]
assign labels:  [[32. 23. 35

image test number:  11850
actual labels:  [[13 14 24]]
assign labels:  [[13. 24. 14. 23. 26.]]
image test number:  11851
actual labels:  [[ 0  7 31 32]]
assign labels:  [[23. 22.  7. 35. 24.]]
image test number:  11852
actual labels:  [[ 0  7  8 31 32]]
assign labels:  [[35. 23. 24. 22. 19.]]
image test number:  11853
actual labels:  [[24 27 31 32 33 35 37]]
assign labels:  [[33.  7. 31. 35. 22.]]
image test number:  11854
actual labels:  [[16 29 31 32 34 37]]
assign labels:  [[33. 35. 17. 31.  7.]]
image test number:  11855
actual labels:  [[13 14 24]]
assign labels:  [[14. 13. 24. 35. 31.]]
image test number:  11856
actual labels:  [[24 31 33 35]]
assign labels:  [[33.  7. 35.  8. 31.]]
image test number:  11857
actual labels:  [[16]]
assign labels:  [[22. 23. 18. 11. 16.]]
image test number:  11858
actual labels:  [[24 27 31 35 37]]
assign labels:  [[35. 24. 36. 23. 22.]]
image test number:  11859
actual labels:  [[11 18 22 23 31 32 34]]
assign labels:  [[23. 22. 33. 18. 11.]]
image

image test number:  12152
actual labels:  [[11 16 20 22]]
assign labels:  [[22. 23. 20. 11. 24.]]
image test number:  12153
actual labels:  [[11 12 18 19 22 23 25 26 32]]
assign labels:  [[23. 19. 22. 18. 26.]]
image test number:  12154
actual labels:  [[ 7 17 29 31 32 33 37]]
assign labels:  [[17. 27. 33.  7. 37.]]
image test number:  12155
actual labels:  [[13 14 24]]
assign labels:  [[13. 14. 24. 22.  0.]]
image test number:  12156
actual labels:  [[ 0  3  4 24 32]]
assign labels:  [[24.  3.  4. 35.  0.]]
image test number:  12157
actual labels:  [[32]]
assign labels:  [[23. 19. 22. 12. 18.]]
image test number:  12158
actual labels:  [[11 12 22 32]]
assign labels:  [[22. 35. 23. 18. 24.]]
image test number:  12159
actual labels:  [[18 19 22 23 24 31 33]]
assign labels:  [[23. 22. 26. 25. 19.]]
image test number:  12160
actual labels:  [[16]]
assign labels:  [[22. 16. 11. 23. 18.]]
image test number:  12161
actual labels:  [[0]]
assign labels:  [[23. 22. 25. 11. 12.]]
image test numb

assign labels:  [[33. 35.  7. 29. 31.]]
image test number:  12441
actual labels:  [[20 21 27 31 32 37]]
assign labels:  [[33. 21. 20. 31. 35.]]
image test number:  12442
actual labels:  [[11 12 16 22 23]]
assign labels:  [[23. 12. 22. 11. 26.]]
image test number:  12443
actual labels:  [[24 29 31 32 34 37]]
assign labels:  [[33. 35.  7. 23. 31.]]
image test number:  12444
actual labels:  [[13 14 24]]
assign labels:  [[23. 22. 12. 13. 14.]]
image test number:  12445
actual labels:  [[13 14 24]]
assign labels:  [[13. 14. 24. 22. 23.]]
image test number:  12446
actual labels:  [[ 0  3  4  7  8 18 19 22 23 31 32]]
assign labels:  [[33.  7.  8. 35. 17.]]
image test number:  12447
actual labels:  [[ 0  3  4  7 27 29 31 32 33 34 37]]
assign labels:  [[29. 23. 33. 22. 19.]]
image test number:  12448
actual labels:  [[ 0  9 10 24]]
assign labels:  [[35. 24. 23. 22. 17.]]
image test number:  12449
actual labels:  [[32]]
assign labels:  [[35. 22. 18. 24. 23.]]
image test number:  12450
actual lab